Based on: http://lethain.com/genetic-algorithms-cool-name-damn-simple/

Function to solve: http://www.pi.zarz.agh.edu.pl/tematy/EA/labor/ea_zad2.html

In [3]:
from random import randint, random
from operator import add
from functools import reduce

#### Create member of population and population 

In [4]:
def individual(length, min, max):
     return [ randint(min,max) for x in range(length) ]

In [5]:
def population(count, length, min, max):
    return[individual(length, min, max) for x in range(count)]

#### Fitness function 

In [100]:
def fitness(individual):
    fx=100*pow((pow(individual[0]*0.001,2))-individual[1]*0.001,2)
    +pow((1-individual[0]*0.001),2)
    return fx  #liczba ujemna mniejsza 

#### Population parameters 

In [105]:
def grade(pop): #średni błąd dla populacji
    summed= reduce(add, (fitness(x) for x in pop),0)
    return(summed/(len(pop)*1.0))

## Breeding & mutation rules 

In [161]:
def evolve(pop, retain=0.1, mutate=0.03):
    graded = [ (fitness(x), x) for x in pop] #fit function and indifidual for each individual
    graded = [ x[1] for x in sorted(graded)] #above, sorted by fytness (lowest to highest)
    retain_length = int(len(graded)*retain) #select indifiduals to stay unchaanged
    parents = graded[:retain_length] # take it to crossover

    # mutate some individuals
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            # this mutation is not ideal, because it
            # restricts the range of possible values,
            # but the function is unaware of the min/max
            # values used to create the individuals,
            individual[pos_to_mutate] = randint(
                min(individual), max(individual))

    # crossover parents to create children
    parents_length = len(parents)  # take 2 random elements and create child, may not choose some elements, 
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = len(male) / 2
            child = male[:int(half)] + female[int(half):]
            children.append(child)

    parents.extend(children)
    return parents

## Target


In [163]:
p_count = 200
i_length = 2
i_min = -2047
i_max = 2047
p = population(p_count, i_length, i_min, i_max)
fitness_history = [grade(p),]
for i in range(100):
    populacja = p
    p = evolve(p)
    fitness_history.append(grade(p))
    
for datum in fitness_history:
    print (datum)
print("population: \n" ,populacja)

528.2926855476109
65.47674919847553
44.86846043321251
26.8959594676275
0.006681737412500122
0.006522177600000159
0.839163346960002
0.27353939359999946
1.0127455977600024
0.006522177600000159
5.449488825599994
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
2.009074550694003
0.006522177600000159
0.006522177600000159
0.5169827730300003
0.006522177600000159
0.006522177600000159
0.9042029616000029
0.7149180176000022
0.006522177600000159
0.7648679616000025
1.2971026576000029
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
1.9556020656000037
4.069592097600004
0.006522177600000159
0.006522177600000159
0.006522177600000159
1.0907826736000026
0.006522177600000159
0.006522177600000159
0.006522177600000159
0.006522177600000159
2.7236691089675027
6.114640193599968
0.25393378559999996
0.2069467718449994
